In [1]:
from experiments.metric import *

In [2]:
import numpy as np
grid = np.meshgrid([0,1,2],["A","B"])

grid = [np.ravel(x) for x in grid]
grid = np.array(grid).T
grid



array([['0', 'A'],
       ['1', 'A'],
       ['2', 'A'],
       ['0', 'B'],
       ['1', 'B'],
       ['2', 'B']], dtype='<U11')

In [44]:
f_groups = Flags.All
print(f_groups)
f_group_numb = len(f_groups)
f_grouped = [f.All for f in f_groups]

metrics_numb = np.prod([len(f) for f in f_grouped])
lookup = np.empty((f_group_numb+1,metrics_numb), dtype=object)
print(lookup.shape)

grid = np.meshgrid(*f_grouped)
grid = [np.ravel(x) for x in grid]
grid = np.array(grid)
print(grid.shape)
grid = np.array(grid)
indxs = np.arange(metrics_numb)
grid = np.vstack([grid,indxs])
grid[0]

[_DistrFlags, _LevelFlags, _MetricFlags, _PredictionFlags]
(5, 48)
(4, 48)


array([InLen, InLen, InLen, InLen, InLen, InLen, InLen, InLen, OutLen,
       OutLen, OutLen, OutLen, OutLen, OutLen, OutLen, OutLen, Avrg, Avrg,
       Avrg, Avrg, Avrg, Avrg, Avrg, Avrg, InLen, InLen, InLen, InLen,
       InLen, InLen, InLen, InLen, OutLen, OutLen, OutLen, OutLen, OutLen,
       OutLen, OutLen, OutLen, Avrg, Avrg, Avrg, Avrg, Avrg, Avrg, Avrg,
       Avrg], dtype=object)

In [8]:
Flags.group_flags(Flags.All).keys()

dict_keys([_DistrFlags, _LevelFlags, _MetricFlags, _PredictionFlags])

In [14]:
d = {1:"A",2:"B"}
for i in d.items():
    print(i)

(1, 'A')
(2, 'B')


In [45]:
lookup = grid
from experiments.metric import MetricTemplate
def find_index(cls, template: MetricTemplate):
        f_grouped = Flags.group_flags(template.flags)
        [f_grouped.update({k:k.All}) if len(v) == 0 else None for k,v in f_grouped.items()]
        print(f_grouped)

        indxs = np.full(lookup.shape[1], True)
        for i,flags_in_group in enumerate(f_grouped.values()):
            hits = np.isin(lookup[i],flags_in_group, assume_unique=True)
            indxs = indxs & hits
        print(indxs)
        return lookup[-1][indxs]

temp = MetricTemplate(None,
                      [Flags.PredictionFlags.NO_ORACLE,
                       Flags.DistrFlags.Avrg,
                       Flags.LevelFlags.SL,

                       ])
indxs = find_index(None,temp)

grid[:,indxs[3]]

{_DistrFlags: [Avrg], _LevelFlags: [SL], _MetricFlags: [CORRECT, TOTAL, ACC, ERR], _PredictionFlags: [NO_ORACLE]}
[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False  True False  True False  True False  True]


array([Avrg, SL, ERR, NO_ORACLE, 47], dtype=object)

In [5]:
l = [1,2,3,4,5]
dict(l)

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [83]:
A = {'a':1,'b':2}
B = {'a':3,'b':4}
C = {"X":A,"Y":B}
D = {"X":A,"Y":B}
E = {"Q":C,"P":D}


# change to
# A = {'Q':1,'P':2}
# B = {'Q':3,'P':4}
# C = {"X":A,"Y":B}
# D = {"X":A,"Y":B}
# E = {"a":C,"b":D}

def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

flatten_dict(E)

def unflatten_dict(d, sep='_'):
    items = {}
    for k, v in d.items():
        keys = k.split(sep)
        sub_items = items
        for key in keys[:-1]:
            sub_items = sub_items.setdefault(key, {})
        sub_items[keys[-1]] = v
    return items


flat = flatten_dict(E)
print(E)

new_flat = {}
for k,v in flat.items():
    ks = k.split("_")
    p = ks.pop(0)
    ks.append(p)
    ks = "_".join(ks)
    new_flat[ks] = v
#[flat.pop(k) for k in old_keys]

unflatten_dict(new_flat)

L = ["A","B","C","D"]
l = [4,3,2,1]


[L[i-1] for i in l]

{'Q': {'X': {'a': 1, 'b': 2}, 'Y': {'a': 3, 'b': 4}}, 'P': {'X': {'a': 1, 'b': 2}, 'Y': {'a': 3, 'b': 4}}}


['D', 'C', 'B', 'A']